In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas  # noqa
import os
import sys
import re
pd.options.plotting.backend = 'holoviews'

## Utils

In [7]:
# https://lowcarbonpower.org/methodology
# Function to replace a substring in a string column
def process_values(value):
    if isinstance(value, str):
        value = re.sub(r'[\r\n]+', ' - ', value).strip('"\' \)\()')

    return np.nan if not value else value


# Clean columns content
def process_columns(column_name):
    # return column_name

    # BASIC
    column_name = re.sub(r'([A-Z]+)', r' \1', column_name.lower())
    column_name = re.sub(r'(\s+|\.)', r' ', column_name).strip()
    column_name = re.sub(r'(\s+)', r'_', column_name)

    # COUNTRY AND REGION
    column_name = re.sub(r'(iso[2-3])code', r'\1', column_name)
    column_name = re.sub(r'(country)(iso[2-3])', r'\1_\2', column_name)
    column_name = re.sub(r'(country)(region)', r'\2', column_name)
    column_name = re.sub(r'(?<=region)(?=type|name)', '_', column_name)
    column_name = re.sub(r'(?<=region)(?=id)', '_', column_name)
    column_name = re.sub(r'(?<=type)(?=id|name)', '_', column_name)

    column_name = column_name.replace('country_code', 'country_iso3')

    # DATE RELATED
    column_name = re.sub(r'(?<=last)(?=updated)', '_', column_name)

    return column_name

# FORMAT DF:
def process_df(df):
    # PROCESSING
    # STRING VALUES (might set to NaN)
    df = df.applymap(process_values)
    # Uniformize NaN values
    df = df.replace(['', 'nan'], np.nan)

    # Drop all columns having only NaN
    df = df.dropna(axis=1, how='all')

    # Replace column names using the format function
    df = df.rename(columns=process_columns)

    # Filter value if exists
    if ('value' in df.columns):
        df = df.dropna(subset=['value'])
    
    # Change year type to int
    if ('year' in df.columns):
        df['year'] = df['year'].astype(int)   

    return df

## Load and Process Data

### all data

In [8]:
df_iea = pd.read_csv("../../../data/_raw/iea/iea_api_eei.csv")
df_iea = process_df(df_iea)
df_iea.head()

,country,tab,flow,enduse,activity,unit,year,value
69,Australia,Freight transport,SHARE_COUNTRY_EM,Freight trains,NaN,%,2013,0.90
70,Australia,Freight transport,SHARE_COUNTRY_EM,Trucks,NaN,%,2013,8.65
71,Australia,Freight transport,SHARE_FOSSIL,Domestic freight ships,NaN,%,2013,88.52
72,Australia,Freight transport,SHARE_FOSSIL,Freight trains,NaN,%,2013,94.43
73,Australia,Freight transport,SHARE_FOSSIL,Trucks,NaN,%,2013,99.21


In [9]:
df_info = pd.read_csv("../../../data/_info/__INFO_UN_M49_en.csv")
df_info = process_df(df_info)
df_info.head()

,global_code,global_name,region_code,region_name,sub-region_code,sub-region_name,intermediate_region_code,intermediate_region_name,country_or_area,m49_code,iso-alpha2_code,iso-alpha3_code,least_developed_countries_(ldc),land_locked_developing_countries_(lldc),small_island_developing_states_(sids)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN


In [10]:
# Mapping data with info_m49_en

# Left joining
df_merged = df_iea.merge(df_info, left_on='country', right_on='country_or_area', how='left')

# Filter columns
selected_columns = ['country', 'iso-alpha3_code', 'year', 'tab', 'flow', 'enduse', 'region_name', 'sub-region_name', 'activity', 'unit', 'value']
df_merged = df_merged[selected_columns]
df_merged.head()

,country,iso-alpha3_code,year,tab,flow,enduse,region_name,sub-region_name,activity,unit,value
0,Australia,AUS,2013,Freight transport,SHARE_COUNTRY_EM,Freight trains,Oceania,Australia and New Zealand,NaN,%,0.90
1,Australia,AUS,2013,Freight transport,SHARE_COUNTRY_EM,Trucks,Oceania,Australia and New Zealand,NaN,%,8.65
2,Australia,AUS,2013,Freight transport,SHARE_FOSSIL,Domestic freight ships,Oceania,Australia and New Zealand,NaN,%,88.52
3,Australia,AUS,2013,Freight transport,SHARE_FOSSIL,Freight trains,Oceania,Australia and New Zealand,NaN,%,94.43
4,Australia,AUS,2013,Freight transport,SHARE_FOSSIL,Trucks,Oceania,Australia and New Zealand,NaN,%,99.21


In [11]:
df_merged['flow'].unique()

array(['SHARE_COUNTRY_EM', 'SHARE_FOSSIL', 'E_FINAL', 'ENERGY_EFF_CARBON',
       'ENERGY_EFF_ENERGY'], dtype=object)

### Final energy

In [12]:
df_final_energy = df_merged[df_merged['flow']=='E_FINAL']
df_final_energy = df_final_energy.drop(columns=['activity'])
df_final_energy = df_final_energy.groupby(['country', 'iso-alpha3_code', 'year', 'tab', 'region_name', 'sub-region_name', 'unit'])['value'].sum().reset_index()
df_final_energy["source"] = 'iea'
df_final_energy["type"] = 'final_energy'
df_final_energy.rename(columns={'iso-alpha3_code':'country_code_a3', 'tab':'sector', 'sub-region_name':'subregion_name'}, inplace=True)
df_final_energy.to_csv('../../../data/_processed/iea_final_energy.csv', index=False)
df_final_energy.head()

,country,country_code_a3,year,sector,region_name,subregion_name,unit,value,source,type
0,Albania,ALB,2000,Industry and services,Europe,Southern Europe,PJ,25.12,iea,final_energy
1,Albania,ALB,2000,Passenger transport,Europe,Southern Europe,PJ,20.14,iea,final_energy
2,Albania,ALB,2000,Residential,Europe,Southern Europe,PJ,15.75,iea,final_energy
3,Albania,ALB,2001,Industry and services,Europe,Southern Europe,PJ,25.65,iea,final_energy
4,Albania,ALB,2001,Passenger transport,Europe,Southern Europe,PJ,21.17,iea,final_energy


### ENERGY_EFF_CARBON

In [13]:
df_energy_eff_carbon = df_merged[df_merged['flow']=='ENERGY_EFF_CARBON']
df_energy_eff_carbon.head()

,country,iso-alpha3_code,year,tab,flow,enduse,region_name,sub-region_name,activity,unit,value
8,Australia,AUS,2014,Freight transport,ENERGY_EFF_CARBON,Domestic freight ships,Oceania,Australia and New Zealand,NaN,gCO2/tkm,6.6
9,Australia,AUS,2014,Freight transport,ENERGY_EFF_CARBON,Freight trains,Oceania,Australia and New Zealand,NaN,gCO2/tkm,9.2
10,Australia,AUS,2014,Freight transport,ENERGY_EFF_CARBON,Trucks,Oceania,Australia and New Zealand,NaN,gCO2/tkm,164.2
23,Australia,AUS,2015,Freight transport,ENERGY_EFF_CARBON,Domestic freight ships,Oceania,Australia and New Zealand,NaN,gCO2/tkm,6.8
24,Australia,AUS,2015,Freight transport,ENERGY_EFF_CARBON,Freight trains,Oceania,Australia and New Zealand,NaN,gCO2/tkm,8.8


* 'gCO2/pkm' : CO2 emissions per passenger kilometer travelled (pkm) for new cars
* 'gCO2/tkm' : per ton kilometer travelled (tkm) for new trucks (between two dates of measurement of the indicator).

In [14]:
df_energy_eff_carbon['unit'].unique()

array(['gCO2/tkm', 'gCO2/USD', 'gCO2/pkm', 'kgCO2/capita',
       'kgCO2/dwelling', 'kgCO2/m2'], dtype=object)

In [15]:
df_energy_eff_carbon[df_energy_eff_carbon['unit']=='kgCO2/m2']['enduse'].unique()

array(['Lighting', 'Space cooling', 'Space heating'], dtype=object)

In [16]:
dd = df_energy_eff_carbon.groupby(['year','country', 'unit'])['value'].sum().reset_index()

In [17]:
ddd = dd[(dd['unit']=='gCO2/USD')&(dd['country']=='France')]
ddd

,year,country,unit,value
50,2000,France,gCO2/USD,6069.6
239,2001,France,gCO2/USD,6055.4
429,2002,France,gCO2/USD,6206.0
619,2003,France,gCO2/USD,6251.0
809,2004,France,gCO2/USD,6258.6
1006,2005,France,gCO2/USD,6736.4
1203,2006,France,gCO2/USD,6997.2
1404,2007,France,gCO2/USD,6699.8
1604,2008,France,gCO2/USD,6322.5
1807,2009,France,gCO2/USD,4809.7


In [19]:
dd.hvplot(x='year', by=['country'], groupby =['country', 'unit'], widget_location='left_top', title='Total emission of CO2eq(Mtonne)')

Row
    [0] WidgetBox
        [0] Select(margin=(20, 20, 5, 20), name='country', options=['Argentina', ...], value='Argentina', width=250)
        [1] Select(margin=(5, 20, 20, 20), name='unit', options=['gCO2/USD', 'kgCO2/capita...], value='gCO2/USD', width=250)
    [1] HoloViews(DynamicMap, height=300, sizing_mode='fixed', widget_location='left_top', width=700)

In [28]:
df = pd.read_csv(url)

HTTPError: HTTP Error 404: Not Found

In [27]:
url = "https://www.iea.org/product/download/014651-000281-014379/IEA - Energy End-uses and Efficiency Indicators database - Highlights (June 2023).xlsb"
url = url.replace(" ", "_")
url

'https://www.iea.org/product/download/014651-000281-014379/IEA_-_Energy_End-uses_and_Efficiency_Indicators_database_-_Highlights_(June_2023).xlsb'

In [29]:
df_final_energy['region_name'].unique()

array(['Europe', 'Americas', 'Oceania', 'Asia', 'Africa'], dtype=object)

In [30]:
df_final_energy['subregion_name'].unique()

array(['Southern Europe', 'Latin America and the Caribbean',
       'Australia and New Zealand', 'Western Europe', 'Western Asia',
       'Eastern Europe', 'Northern America', 'Northern Europe',
       'Eastern Asia', 'Northern Africa', 'Sub-Saharan Africa'],
      dtype=object)